In [1]:
from googletrans import Translator

target_texts = ['안녕하세요', '밥 먹었습니까?']

translator = Translator()
translated_texts = []

for target_text in target_texts:
    translated_texts.append(translator.translate(text=target_text, dest='en').text)
    
translated_texts

['Hello', 'Did you have some meal?']

In [2]:
DELIMITER = '\n'
MAX_BYTE = 1024 * 15  # 15k

In [3]:
def wrap_text(texts):
    """
    Request a section that is not translated by combining sentences with specific symbols.
    ex) ['안녕하세요', '밥 먹었습니까?'] → '안녕하세요\n밥 먹었습니까?'
    """
    texts = [text.replace(DELIMITER, '<span class="notranslate">space</span>') for text in texts]
    return DELIMITER.join(texts)

def unwrap_text(wrapped_text):
    """
    Cuts the translated sentence back to a specific symbol.
    예) '안녕하세요\n밥 먹었습니까?' →  ['안녕하세요', '밥 먹었습니까?']
    """
    texts = wrapped_text.split(DELIMITER)
    texts = [text.replace('<span class="notranslate">space</span>', DELIMITER) for text in texts]
    return texts

In [4]:
wrap_text(target_texts)

'안녕하세요\n밥 먹었습니까?'

In [5]:
def chunk_text_byte(text, max_byte=MAX_BYTE, delimiter=DELIMITER):
    """
    When the translate function is sequentially requested for a large number of translated sentences, 
    it internally requests the API request several times, resulting in a slow response in terms of performance.
    Since the number of characters is limited up to 15K, it is possible to reduce the network load by requesting in 15K increments.
    """
    text = text.encode('utf-8')
    delimiter = delimiter.encode('utf-8')
    words = iter(text.split(delimiter))
    lines, current = [], next(words)

    for word in words:
        if len(current) + 1 + len(word) > max_byte:
            lines.append(current.decode('utf-8'))
            current = word
        else:
            current += delimiter + word
    lines.append(current.decode('utf-8'))
    return lines

In [6]:
translator = Translator()
translation = ''
for splitted_text in chunk_text_byte(wrap_text(target_texts)):
    translation += translator.translate(text=splitted_text, dest='en').text
    
translated_texts = []
for translated_text in unwrap_text(translation):
    translated_texts.append(translated_text)    
    
print(translated_texts)    

['Hello', 'Did you have some meal?']
